1. Split frmer big function to singles

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import scipy.io
import os


In [25]:
## Read all '.mat' data in one Subject
data_root = '/home/jinyanwei/Documents/JYW_COCHE/Project/MMBP/data/MMData'

'''Sub_combined_data_path = '/home/jinyanwei/Documents/JYW_COCHE/Project/MMBP/data/Sub_combined_data'
if not os.path.exists(Sub_combined_data_path):
    os.makedirs(Sub_combined_data_path)'''
    
## Dict Sub contains all subjects
Sub = {} 
Sub['sub_id'] = list(os.walk(data_root))[0][1] #len(Sub['sub_id']) = 15

def combine_all_subjects_row_data(i):
    Sub[f"{Sub['sub_id'][i]}"] = {}
    rest1_s = []
    CT_s = []
    recover_s = []
    rest2_s = []
    Sub[f"{Sub['sub_id'][i]}"]['seg'] = [n.split('.')[0] for n in list(os.walk(data_root))[i+1][2]]
    #print(Sub[f"{Sub['sub_id'][i]}"]['seg'])
    for s in Sub[f"{Sub['sub_id'][i]}"]['seg']:
        Sub[f"{Sub['sub_id'][i]}"][s] = scipy.io.loadmat(f"{data_root}/{Sub['sub_id'][i]}/{s}.mat")[s]
        for r in s.split('_'):
            if r in ['recover', 're', 'Recover']: #this seg is one of Recover
                recover_s.append(s)
            elif r in ['rest1']:
                rest1_s.append(s)
            elif r in ['CT', 'ct']:
                CT_s.append(s)
            elif r in ['rest2']:
                rest2_s.append(s)
            elif r in ['rest']:
                rest_index = s.split('_').index('rest')
                if s.split('_')[rest_index + 1] == '1':
                    rest1_s.append(s)
                else:
                    rest2_s.append(s)

    #print(rest1_s)
    ## Type 4 stage in one Subject
    Sub[f"{Sub['sub_id'][i]}"]['rest1'] = np.concatenate([Sub[f"{Sub['sub_id'][i]}"][si] for si in rest1_s], axis=1)
    r1code = np.full((1, Sub[f"{Sub['sub_id'][i]}"]['rest1'].shape[1]), 1)
    Sub[f"{Sub['sub_id'][i]}"]['rest1'] = np.concatenate((Sub[f"{Sub['sub_id'][i]}"]['rest1'], r1code), axis=0)
    Sub[f"{Sub['sub_id'][i]}"]['CT'] = np.concatenate([Sub[f"{Sub['sub_id'][i]}"][si] for si in CT_s], axis=1)
    CTcode = np.full((1, Sub[f"{Sub['sub_id'][i]}"]['CT'].shape[1]), 2)
    Sub[f"{Sub['sub_id'][i]}"]['CT'] = np.concatenate((Sub[f"{Sub['sub_id'][i]}"]['CT'], CTcode), axis=0)
    Sub[f"{Sub['sub_id'][i]}"]['recover'] = np.concatenate([Sub[f"{Sub['sub_id'][i]}"][si] for si in recover_s], axis=1)
    recode = np.full((1, Sub[f"{Sub['sub_id'][i]}"]['recover'].shape[1]), 3)
    Sub[f"{Sub['sub_id'][i]}"]['recover'] = np.concatenate((Sub[f"{Sub['sub_id'][i]}"]['recover'], recode), axis=0)
    Sub[f"{Sub['sub_id'][i]}"]['rest2'] = np.concatenate([Sub[f"{Sub['sub_id'][i]}"][si] for si in rest2_s], axis=1)
    r2code = np.full((1, Sub[f"{Sub['sub_id'][i]}"]['rest2'].shape[1]), 4)
    Sub[f"{Sub['sub_id'][i]}"]['rest2'] = np.concatenate((Sub[f"{Sub['sub_id'][i]}"]['rest2'], r2code), axis=0)
    ## Concat to on entire one
    Sub[f"{Sub['sub_id'][i]}"]['data'] = np.concatenate((Sub[f"{Sub['sub_id'][i]}"]['rest1'], Sub[f"{Sub['sub_id'][i]}"]['CT'], Sub[f"{Sub['sub_id'][i]}"]['recover'], Sub[f"{Sub['sub_id'][i]}"]['rest2']), axis=1)
    #print(len(Sub[f"{Sub['sub_id'][i]}"]['data'][0]))
    #print(Sub[f"{Sub['sub_id'][i]}"]['data'])
    '''sub_comb_filename = Sub_combined_data_path + f"/S{Sub['sub_id'][i]}_combined.csv"
    np.savetxt(sub_comb_filename, Sub[f"{Sub['sub_id'][i]}"]['data'], delimiter=',')'''
    
    return

## Get 5 signals and show
fs = 2000
tags = ['ECG', 'CBP', 'PPG', 'IPG', 'Temp', 'Stage', 'ECGf']
result_root = '/home/jinyanwei/Documents/JYW_COCHE/Project/MMBP/result/MMData'

def get_signals_show(i, figshow=0):
    for da in range(6):
        Sub[f"{Sub['sub_id'][i]}"][f"{tags[da]}"] = Sub[f"{Sub['sub_id'][i]}"]['data'][da]
    if figshow==1:
        begin_index, end_index = 0, 30000 #S1.shape[1]
        fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(18, 10), dpi=96)
        for sgn in range(5):
            axs[sgn].plot(Sub[f"{Sub['sub_id'][i]}"]['data'][sgn][begin_index:end_index])
            axs[sgn].set_title(f"{tags[sgn]}")
        fig.subplots_adjust(hspace=0.55)
        result_path = result_root + f"/S{Sub['sub_id'][i]}"
        if not os.path.exists(result_path):
            os.makedirs(result_path)
        plt.savefig(f"{result_path}/S{Sub['sub_id'][i]}_{int(begin_index/fs)}to{int(end_index/fs)}sec.png")
        plt.show()

    return

for i in range(len(Sub['sub_id'])):
    combine_all_subjects_row_data(i)
    get_signals_show(i)


In [26]:
## Filter ECG and show